# Computer Vision

## Assignment 3

Vedansh Agrawal <br>
2021112010

## Question 1: Face detection and association-based tracking

### Part 1:

We downloaded the movie Forrest gump and converted the first 30s into 720 frames with 24fps.

### Part 2:

In this part of the assignment we need to use Viola-Jones detector to detect faces from a 30 second clip from the movie Forrest Gump

In [1]:
import cv2
import os
import time
from tqdm import tqdm
import numpy as np
from ultralytics import YOLO

In [2]:
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

def detect_faces(frame):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.2, minNeighbors=5, minSize=(10, 10))
    return faces

cv2_data_dir = cv2.data.haarcascades
xml_file_path = os.path.join(cv2_data_dir, 'haarcascade_frontalface_default.xml')
with open(xml_file_path, 'r') as file:
    xml_contents = file.read()

with open('xml_config.txt', 'w') as file:
    file.write(xml_contents)

In [3]:
for img in tqdm(os.listdir('Frames')):
    start_time = time.time()
    frame = cv2.imread(os.path.join('Frames', img))
    faces = detect_faces(frame)

    for (x, y, w, h) in faces:
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)

    cv2.imwrite(f'Frames_with_faces/{img}', frame)
    end_time = time.time()

cv2.destroyAllWindows()
print(f"Time taken to process a frame is: {end_time - start_time}s")

FileNotFoundError: [Errno 2] No such file or directory: 'Frames'

It took 0.07 seconds to process one frame. 

The factors which influence the time it would take to process another frame, is
- Frame Size
- Scale Factor
- Minimum size
- Minimum behaviour
- Haar Cascade

### Part 3:

Now from the frames generated let us stitch them together to make the video.

In [4]:
output_video = 'Faces.mp4'
frame_rate = 24

images = sorted([img for img in os.listdir('Frames_with_faces')])

first_img = cv2.imread(os.path.join('Frames_with_faces', images[0]))
h, w, _ = first_img.shape

video_writer = cv2.VideoWriter(output_video, cv2.VideoWriter_fourcc(*'mp4v'), frame_rate, (w, h))

for image in images:
    frame = cv2.imread(os.path.join('Frames_with_faces', image))
    video_writer.write(frame)

video_writer.release()

Link to Video:
https://drive.google.com/file/d/1lYEv599SQWqsDuVJuZZUhj2urI_H6nhb/view?usp=sharing

Conclusions from the result:
- It faces difficulty in identifying faces far away from the camera, that is very small faces in the frame.
- It fails when the only half the face is visible such as positions where the character is not facing the camera.
- There are some false positives where the detector detects parts of fabrics or empty spaces as faces.

Some times the window sizes arent small enough to detect the faces far away, and when half faces are shown, the filters face difficulty in identfying it as a face.

### Part 4:

Now we need to implement association based tracking.

In [5]:
def get_iou(face1, face2):
    x1, y1, w1, h1 = face1
    x2, y2, w2, h2 = face2
    x_intersection = max(x1, x2)
    y_intersection = max(y1, y2)
    w_intersection = max(0, min(x1 + w1, x2 + w2) - x_intersection)
    h_intersection = max(0, min(y1 + h1, y2 + h2) - y_intersection)
    intersection_area = w_intersection * h_intersection
    union_area = w1 * h1 + w2 * h2 - intersection_area
    iou = intersection_area / union_area

    return iou

In [6]:
frames = sorted([img for img in os.listdir('Frames')])
tracks = []
track_id_counter = 0

for i in tqdm(range(len(frames) - 1)):
    frame1 = cv2.imread(os.path.join('Frames', frames[i]))
    frame2 = cv2.imread(os.path.join('Frames', frames[i+1]))
    
    faces1 = detect_faces(frame1)
    faces2 = detect_faces(frame2)

    for face1 in faces1:
        max_iou = 0
        associated_face = None

        for face2 in faces2:
            Iou = get_iou(face1, face2)

            if Iou > max_iou:
                max_iou = Iou
                associated_face = face2

        x, y, w, h = face1
        cv2.rectangle(frame1, (x, y), (x+w, y+h), (0, 255, 0), 2)
        for track in tracks:
            if np.all(track['frames'][-1] == face1):
                cv2.putText(frame1, str(track['id']), (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
                break


        if max_iou > 0.2:
            for track in tracks:
                if np.all(track['frames'][-1] == face1):
                    track['frames'].append(associated_face)
                    break
            else:
                track_id_counter += 1
                tracks.append({'id': track_id_counter, 'frames': [face1, associated_face]})

            x, y, w, h = face1
            cv2.rectangle(frame1, (x, y), (x+w, y+h), (0, 255, 0), 2)
            for track in tracks:
                if np.all(track['frames'][-2] == face1):
                    cv2.putText(frame1, str(track['id']), (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
                    break
        else:
            for track in tracks:
                if np.all(track['frames'][-1] == face1):
                    track['finished'] = True

            x, y, w, h = face1
            cv2.rectangle(frame1, (x, y), (x+w, y+h), (0, 255, 0), 2)
            for track in tracks:
                if np.all(track['frames'][-1] == face1):
                    cv2.putText(frame1, str(track['id']), (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
                    break


    tracks = [track for track in tracks if not track.get('finished', False)]
        

    cv2.imwrite(f'Frames_with_faces_association/{frames[i]}', frame1)


cv2.destroyAllWindows()
print('Number of unique tracks:',track_id_counter)

100%|██████████| 719/719 [03:30<00:00,  3.41it/s]

Number of unique tracks: 27


Now we need to stitch all the images together.

In [7]:
output_video = 'Faces_with_association.mp4'
frame_rate = 15

images = sorted([img for img in os.listdir('Frames_with_faces_association')])

first_img = cv2.imread(os.path.join('Frames_with_faces_association', images[0]))
h, w, _ = first_img.shape

video_writer = cv2.VideoWriter(output_video, cv2.VideoWriter_fourcc(*'mp4v'), frame_rate, (w, h))

for image in images:
    frame = cv2.imread(os.path.join('Frames_with_faces_association', image))
    video_writer.write(frame)

video_writer.release()

Link to Video:
https://drive.google.com/file/d/1lg1s6BG0I2w90b8v7X85cmeA6pinZC6z/view?usp=sharing

### Quality of the Associations:

In the associations made from the above code, no 2 characters got associated to the same track, each track had only one character being detected. But one character does not have one unique id. There are often breaks in the track because there may be frames where the face is not detected by the detector in which case the track breaks and start a new track in the subsequent frames. This could be resolved if we not just compare the faces detected in adjacent frames rather in a given window of frames. 

## Question 2: YOLO Object Detection

### Part 1:

In this part we are required to install the dataset from the kaggle link given to us. We note how the data is stored as images and targets and inside both files we have images in files named as train and test.

### Part 2: Understanding YOLO

YOLO is an object detection algorithm which is one shot unline the R-CNN which is a 2 shot object detctor. The difference is that YOLO uses a single pass of the input image to make predictions about the presence and location of objects in the image. This makes them computationally more efficient with a trade off for accuracy and efficiency. YOLO has a hard time detecting smaller objects. On the other hand, we have R-CNN which is 2-shot which means the image goes through 2 passes of the CNN, one to obtain region proposal areas and the second time to process these RoIs and make predictions. The algorithm for YOLO v1 involves dividing the image into a sxs grid and then performing a prediction on each cell. The cell containg the centre of a bounding box is responsible for detecting that image in the cell. We use IoU to remove most bouding boxes which detect the same object and then we use a post processing method called non-maximum suppresion to get rid of even more boxes and make the predictions more accurate. While this first version used a VGG net as its CNN, the YOLOv2 uses Darknet-19. Additionally the biggest change made in YOLO v2 was the adition of anchor boxes, which are a set of predefined bounding boxes of different aspect ratios and scales. When predicting bounding boxes, YOLOv2 uses a combination of the anchor boxes and the predicted offsets to determine the final bounding box. This allows the algorithm to handle a wider range of object sizes and aspect ratios. Then came the YOLOv3 which uses the Darknet-53 architecture which is a variant of the ResNet.In YOLO v3 the anchor boxes are scaled, and aspect ratios are varied to better match the size and shape of the objects being detected, whereas in YOLOv2, the anchor boxes were all the same size which limited the ability of the detector. Additionally, the YOLOv3 introduces feature pyramid networks, which creates a pyramid of feature maps, with each level of the pyramid being used to detect objects at a different scale.This helps to improve the detection performance on small objects, as the model is able to see the objects at multiple scales.

### Part 3: 

Now let us load the nano and the medium model and compare the iszes of the 2 models.

In [8]:
model_nano_100 = YOLO('yolov8n.yaml')
model_nano_400 = YOLO('yolov8n.yaml')
model_nano_100.info()

YOLOv8n summary: 225 layers, 3157200 parameters, 3157184 gradients, 8.9 GFLOPs


(225, 3157200, 3157184, 8.8575488)

In [9]:
model_nano_pretrained_100 = YOLO('yolov8n.pt')
model_nano_pretrained_400 = YOLO('yolov8n.pt')
model_nano_pretrained_100.info()

YOLOv8n summary: 225 layers, 3157200 parameters, 0 gradients, 8.9 GFLOPs


(225, 3157200, 0, 8.8575488)

In [10]:
model_medium_100 = YOLO('yolov8m.pt')
model_medium_400 = YOLO('yolov8m.pt')
model_medium_100.info()

YOLOv8m summary: 295 layers, 25902640 parameters, 0 gradients, 79.3 GFLOPs


(295, 25902640, 0, 79.3204224)

As we can see the number of parameters in the medium model is significantly higher than the nano model and the number of convolutional layers is more in the medium model than the nano model.

### Part 4:

In this part we need to train the above 3 models for 2 different train datasets.

In [11]:
model_nano_400.train(data='YOLO_dataset/config.yaml', epochs = 20)
#AP=24.62%

New https://pypi.org/project/ultralytics/8.1.44 available 😃 Update with 'pip install -U ultralytics'


/home/vedansh/.local/lib/python3.8/site-packages/torch/cuda/__init__.py:107: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 10010). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:109.)
  return torch._C._cuda_getDeviceCount() > 0


Ultralytics YOLOv8.1.43 🚀 Python-3.8.10 torch-2.0.1+cu117 CPU (11th Gen Intel Core(TM) i5-11320H 3.20GHz)
engine/trainer: task=detect, mode=train, model=yolov8n.yaml, data=YOLO_dataset/config.yaml, epochs=20, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train12, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, 

wandb: Currently logged in as: vedu-agrawal (music-cognition). Use `wandb login --relogin` to force relogin
wandb: wandb version 0.16.6 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
wandb: Tracking run with wandb version 0.15.12
wandb: Run data is saved locally in /home/vedansh/Desktop/Computer Vision/Assignment 4/wandb/run-20240407_161201-10x5waks
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run train12
wandb: ⭐️ View project at https://wandb.ai/music-cognition/YOLOv8
wandb: 🚀 View run at https://wandb.ai/music-cognition/YOLOv8/runs/10x5waks


Freezing layer 'model.22.dfl.conv.weight'


...


20 epochs completed in 0.632 hours.
Optimizer stripped from runs/detect/train12/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/train12/weights/best.pt, 6.2MB

Validating runs/detect/train12/weights/best.pt...
Ultralytics YOLOv8.1.43 🚀 Python-3.8.10 torch-2.0.1+cu117 CPU (11th Gen Intel Core(TM) i5-11320H 3.20GHz)
YOLOv8n summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:05<00:00,  2.67s/it]


                   all         50         52      0.367      0.288      0.246     0.0715
Speed: 1.0ms preprocess, 83.1ms inference, 0.0ms loss, 0.6ms postprocess per image
Results saved to runs/detect/train12


wandb: Waiting for W&B process to finish... (success).
wandb: 
wandb: Run history:
wandb:                  lr/pg0 ▃▅▆██▇▇▆▆▅▅▅▄▄▃▃▂▂▁▁
wandb:                  lr/pg1 ▃▅▆██▇▇▆▆▅▅▅▄▄▃▃▂▂▁▁
wandb:                  lr/pg2 ▃▅▆██▇▇▆▆▅▅▅▄▄▃▃▂▂▁▁
wandb:        metrics/mAP50(B) ▁▁▁▁▁▁▁▂▂▂▃▂▃▃▃▂▄▇▇█
wandb:     metrics/mAP50-95(B) ▁▁▁▁▁▁▁▂▂▂▃▂▃▃▃▂▅█▇█
wandb:    metrics/precision(B) ▁▁▁▁▁▁▂█▂▂▃▂▃▃▃▂▃▆▆▇
wandb:       metrics/recall(B) ██▆▇█▁▁▁▄▁▅▃▂▂▁▂▃▄▃▄
wandb:            model/GFLOPs ▁
wandb:        model/parameters ▁
wandb: model/speed_PyTorch(ms) ▁
wandb:          train/box_loss █▇▆▅▅▅▄▄▄▃▃▂▂▂▂▂▁▁▁▁
wandb:          train/cls_loss █▇▇▆▅▅▄▄▃▃▅▄▃▃▂▂▂▁▁▁
wandb:          train/dfl_loss █▇▆▅▄▃▃▃▃▂▂▂▂▂▂▁▁▁▁▁
wandb:            val/box_loss ▆▇██▇▆▆▃▄▄▂▂▂▃▂▄▃▁▁▁
wandb:            val/cls_loss ▃▃▃▃▃█▂▄▂▂▂▄▃▁▂▂▂▁▁▁
wandb:            val/dfl_loss ▆▆██▆▄▃▃▃▂▂▃▁▂▁▂▂▁▁▁
wandb: 
wandb: Run summary:
wandb:                  lr/pg0 0.00012
wandb:                  lr/pg1 0.00012
wandb:                  lr/pg2 0.000

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7fa47a39ebe0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.048048, 

In [12]:
model_nano_pretrained_400.train(data='YOLO_dataset/config.yaml', epochs = 20)
#AP = 72.9%

New https://pypi.org/project/ultralytics/8.1.44 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.1.43 🚀 Python-3.8.10 torch-2.0.1+cu117 CPU (11th Gen Intel Core(TM) i5-11320H 3.20GHz)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=YOLO_dataset/config.yaml, epochs=20, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train13, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, 

wandb: wandb version 0.16.6 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
wandb: Tracking run with wandb version 0.15.12
wandb: Run data is saved locally in /home/vedansh/Desktop/Computer Vision/Assignment 4/wandb/run-20240407_165043-rdvmlkcw
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run train13
wandb: ⭐️ View project at https://wandb.ai/music-cognition/YOLOv8
wandb: 🚀 View run at https://wandb.ai/music-cognition/YOLOv8/runs/rdvmlkcw


Freezing layer 'model.22.dfl.conv.weight'


train: Scanning /home/vedansh/Desktop/Computer Vision/Assignment 4/YOLO_dataset/labels/train.cache... 400 images, 0 backgrounds, 0 corrupt: 100%|██████████| 400/400 [00:00<?, ?it/s]
val: Scanning /home/vedansh/Desktop/Computer Vision/Assignment 4/YOLO_dataset/labels/val.cache... 50 images, 0 backgrounds, 0 corrupt: 100%|██████████| 50/50 [00:00<?, ?it/s]

Plotting labels to runs/detect/train13/labels.jpg... 


...


20 epochs completed in 0.632 hours.
Optimizer stripped from runs/detect/train13/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/train13/weights/best.pt, 6.2MB

Validating runs/detect/train13/weights/best.pt...
Ultralytics YOLOv8.1.43 🚀 Python-3.8.10 torch-2.0.1+cu117 CPU (11th Gen Intel Core(TM) i5-11320H 3.20GHz)
Model summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:05<00:00,  2.98s/it]


                   all         50         52      0.672      0.826      0.729      0.576
Speed: 1.0ms preprocess, 93.9ms inference, 0.0ms loss, 0.5ms postprocess per image
Results saved to runs/detect/train13


wandb: Waiting for W&B process to finish... (success).
wandb: 
wandb: Run history:
wandb:                  lr/pg0 ▃▅▆██▇▇▆▆▅▅▅▄▄▃▃▂▂▁▁
wandb:                  lr/pg1 ▃▅▆██▇▇▆▆▅▅▅▄▄▃▃▂▂▁▁
wandb:                  lr/pg2 ▃▅▆██▇▇▆▆▅▅▅▄▄▃▃▂▂▁▁
wandb:        metrics/mAP50(B) ▆▃▁▅▃▅▃▅▅▄▇▆▇█▇█████
wandb:     metrics/mAP50-95(B) ▆▃▁▃▂▃▃▃▄▃▆▆▇▇▇█▇███
wandb:    metrics/precision(B) ▁▅▃▇▄▅▅▅▅▅█▇▇█▆▇██▇▇
wandb:       metrics/recall(B) █▂▁▃▁▄▁▄▄▂▃▃▅▅▅▅▆▆▆▆
wandb:            model/GFLOPs ▁
wandb:        model/parameters ▁
wandb: model/speed_PyTorch(ms) ▁
wandb:          train/box_loss ▄▄▆▇██▇▇▅▅▆▅▄▄▄▃▂▂▁▁
wandb:          train/cls_loss █▆▆▆▆▅▄▅▄▃▅▄▃▃▂▂▂▂▁▁
wandb:          train/dfl_loss ▄▃▅▇█▇▆▆▄▄▇▆▅▅▅▃▂▂▁▁
wandb:            val/box_loss ▁▄█▅▅▅▅▅▄▅▃▃▂▂▂▂▂▂▂▁
wandb:            val/cls_loss ▄▄▆▅███▄▃▃▂▂▂▁▁▁▁▁▁▁
wandb:            val/dfl_loss ▁▃█▅▆▅▅▄▃▄▂▂▂▂▂▁▂▂▁▁
wandb: 
wandb: Run summary:
wandb:                  lr/pg0 0.00012
wandb:                  lr/pg1 0.00012
wandb:                  lr/pg2 0.000

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7fa4941492e0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.048048, 

In [13]:
model_medium_400.train(data='YOLO_dataset/config.yaml', epochs = 20)
#AP = 70.76%

New https://pypi.org/project/ultralytics/8.1.44 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.1.43 🚀 Python-3.8.10 torch-2.0.1+cu117 CPU (11th Gen Intel Core(TM) i5-11320H 3.20GHz)
engine/trainer: task=detect, mode=train, model=yolov8m.pt, data=YOLO_dataset/config.yaml, epochs=20, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train14, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, 

wandb: wandb version 0.16.6 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
wandb: Tracking run with wandb version 0.15.12
wandb: Run data is saved locally in /home/vedansh/Desktop/Computer Vision/Assignment 4/wandb/run-20240407_172929-q4x4h3sh
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run train14
wandb: ⭐️ View project at https://wandb.ai/music-cognition/YOLOv8
wandb: 🚀 View run at https://wandb.ai/music-cognition/YOLOv8/runs/q4x4h3sh


Freezing layer 'model.22.dfl.conv.weight'


train: Scanning /home/vedansh/Desktop/Computer Vision/Assignment 4/YOLO_dataset/labels/train.cache... 400 images, 0 backgrounds, 0 corrupt: 100%|██████████| 400/400 [00:00<?, ?it/s]
val: Scanning /home/vedansh/Desktop/Computer Vision/Assignment 4/YOLO_dataset/labels/val.cache... 50 images, 0 backgrounds, 0 corrupt: 100%|██████████| 50/50 [00:00<?, ?it/s]


Plotting labels to runs/detect/train14/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.002, momentum=0.9) with parameter groups 77 weight(decay=0.0), 84 weight(decay=0.0005), 83 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 0 dataloader workers
Logging results to runs/detect/train14
Starting training for 20 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...


20 epochs completed in 5.886 hours.
Optimizer stripped from runs/detect/train14/weights/last.pt, 52.0MB
Optimizer stripped from runs/detect/train14/weights/best.pt, 52.0MB

Validating runs/detect/train14/weights/best.pt...
Ultralytics YOLOv8.1.43 🚀 Python-3.8.10 torch-2.0.1+cu117 CPU (11th Gen Intel Core(TM) i5-11320H 3.20GHz)
Model summary (fused): 218 layers, 25840339 parameters, 0 gradients, 78.7 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:22<00:00, 11.25s/it]


                   all         50         52      0.798      0.635      0.708      0.526
Speed: 1.0ms preprocess, 425.6ms inference, 0.0ms loss, 0.3ms postprocess per image
Results saved to runs/detect/train14


wandb: Waiting for W&B process to finish... (success).
wandb: 
wandb: Run history:
wandb:                  lr/pg0 ▃▅▆██▇▇▆▆▅▅▅▄▄▃▃▂▂▁▁
wandb:                  lr/pg1 ▃▅▆██▇▇▆▆▅▅▅▄▄▃▃▂▂▁▁
wandb:                  lr/pg2 ▃▅▆██▇▇▆▆▅▅▅▄▄▃▃▂▂▁▁
wandb:        metrics/mAP50(B) ▃▁▁▁▁▁▂▃▄▆▆▅▇▅▆▇▇███
wandb:     metrics/mAP50-95(B) ▂▁▁▁▁▁▂▂▃▅▅▄▅▄▅▆▇▇██
wandb:    metrics/precision(B) ▃▁▁▁▁▁▂▃▄▆▅▅▇▄█▇▇▇██
wandb:       metrics/recall(B) ▅▇▁▆▆▅▄▅▆▆▆▇▇▇▅▇▇███
wandb:            model/GFLOPs ▁
wandb:        model/parameters ▁
wandb: model/speed_PyTorch(ms) ▁
wandb:          train/box_loss ▁▃▇█▇▇▆▆▅▄▅▅▄▄▃▃▂▂▁▁
wandb:          train/cls_loss ▅▆▇██▇▅▆▅▄▅▅▃▄▄▃▂▂▁▁
wandb:          train/dfl_loss ▁▄▇██▇▆▅▅▄▇▇▅▆▅▄▃▃▂▂
wandb:            val/box_loss ▂▅█▆▇▄▃▃▂▂▂▂▂▂▂▁▁▁▁▁
wandb:            val/cls_loss ▁▁▃▅█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
wandb:            val/dfl_loss ▁▁▁█▄▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
wandb: 
wandb: Run summary:
wandb:                  lr/pg0 0.00012
wandb:                  lr/pg1 0.00012
wandb:                  lr/pg2 0.000

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7fa4943c1100>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.048048, 

In [14]:
model_nano_100.train(data='YOLO_dataset1/config.yaml', epochs = 20)
#AP=2.1%

New https://pypi.org/project/ultralytics/8.1.44 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.1.43 🚀 Python-3.8.10 torch-2.0.1+cu117 CPU (11th Gen Intel Core(TM) i5-11320H 3.20GHz)
engine/trainer: task=detect, mode=train, model=yolov8n.yaml, data=YOLO_dataset1/config.yaml, epochs=20, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train15, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=Non

wandb: wandb version 0.16.6 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
wandb: Tracking run with wandb version 0.15.12
wandb: Run data is saved locally in /home/vedansh/Desktop/Computer Vision/Assignment 4/wandb/run-20240407_232825-wp5rqjv2
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run train15
wandb: ⭐️ View project at https://wandb.ai/music-cognition/YOLOv8
wandb: 🚀 View run at https://wandb.ai/music-cognition/YOLOv8/runs/wp5rqjv2


Freezing layer 'model.22.dfl.conv.weight'


train: Scanning /home/vedansh/Desktop/Computer Vision/Assignment 4/YOLO_dataset1/labels/train.cache... 100 images, 0 backgrounds, 0 corrupt: 100%|██████████| 100/100 [00:00<?, ?it/s]
val: Scanning /home/vedansh/Desktop/Computer Vision/Assignment 4/YOLO_dataset1/labels/val.cache... 50 images, 0 backgrounds, 0 corrupt: 100%|██████████| 50/50 [00:00<?, ?it/s]

Plotting labels to runs/detect/train15/labels.jpg... 


...


20 epochs completed in 0.172 hours.
Optimizer stripped from runs/detect/train15/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/train15/weights/best.pt, 6.2MB

Validating runs/detect/train15/weights/best.pt...
Ultralytics YOLOv8.1.43 🚀 Python-3.8.10 torch-2.0.1+cu117 CPU (11th Gen Intel Core(TM) i5-11320H 3.20GHz)
YOLOv8n summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:04<00:00,  2.37s/it]


                   all         50         52    0.00244      0.615     0.0211    0.00496
Speed: 1.0ms preprocess, 72.4ms inference, 0.0ms loss, 1.4ms postprocess per image
Results saved to runs/detect/train15


wandb: Waiting for W&B process to finish... (success).
wandb: 
wandb: Run history:
wandb:                  lr/pg0 ▁▂▄▅▆▆▇▇█████▇▆▅▄▃▂▁
wandb:                  lr/pg1 ▁▂▄▅▆▆▇▇█████▇▆▅▄▃▂▁
wandb:                  lr/pg2 ▁▂▄▅▆▆▇▇█████▇▆▅▄▃▂▁
wandb:        metrics/mAP50(B) ▃▄▂▂▂▅▂▁▂▃▃▁▂▁▂█▂▂▅█
wandb:     metrics/mAP50-95(B) ▃▄▃▂▂█▂▁▁▂▃▁▁▁▂█▂▂▆▇
wandb:    metrics/precision(B) ▃▃▃▄▃▃▂▃▃▃▃▂▁▂▃▅█▃▅▅
wandb:       metrics/recall(B) ▅▄▄▅▅▅▄▅▇▇▅▄▂▂▄▄▁▄█▄
wandb:            model/GFLOPs ▁
wandb:        model/parameters ▁
wandb: model/speed_PyTorch(ms) ▁
wandb:          train/box_loss █▇▇▆▆▇▆█▆▅▁▄▂▃▂▃▂▁▁▂
wandb:          train/cls_loss ▇▆▅▅▄▅▄▄▂▁▇█▆▆▆▆▆▄▅▅
wandb:          train/dfl_loss █▇▇▇▇▆▆▅▅▄▄▃▃▃▂▂▁▁▁▁
wandb:            val/box_loss ▃▃▃▃▃▂▂▁▁▁▁▅▆█▇▆▇█▇▄
wandb:            val/cls_loss ████▇▆▅▄▂▂▂▃▃▃▄▃▃▃▃▁
wandb:            val/dfl_loss ▃▃▃▃▃▃▂▂▁▁▂▅▆▇▇▆▇█▇▄
wandb: 
wandb: Run summary:
wandb:                  lr/pg0 0.00012
wandb:                  lr/pg1 0.00012
wandb:                  lr/pg2 0.000

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7fa456209370>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.048048, 

In [15]:
model_nano_pretrained_100.train(data='YOLO_dataset1/config.yaml', epochs = 20)
#AP=65.53%

New https://pypi.org/project/ultralytics/8.1.44 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.1.43 🚀 Python-3.8.10 torch-2.0.1+cu117 CPU (11th Gen Intel Core(TM) i5-11320H 3.20GHz)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=YOLO_dataset1/config.yaml, epochs=20, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train16, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None,

wandb: wandb version 0.16.6 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
wandb: Tracking run with wandb version 0.15.12
wandb: Run data is saved locally in /home/vedansh/Desktop/Computer Vision/Assignment 4/wandb/run-20240407_233948-53os2c57
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run train16
wandb: ⭐️ View project at https://wandb.ai/music-cognition/YOLOv8
wandb: 🚀 View run at https://wandb.ai/music-cognition/YOLOv8/runs/53os2c57


Freezing layer 'model.22.dfl.conv.weight'


train: Scanning /home/vedansh/Desktop/Computer Vision/Assignment 4/YOLO_dataset1/labels/train.cache... 100 images, 0 backgrounds, 0 corrupt: 100%|██████████| 100/100 [00:00<?, ?it/s]
val: Scanning /home/vedansh/Desktop/Computer Vision/Assignment 4/YOLO_dataset1/labels/val.cache... 50 images, 0 backgrounds, 0 corrupt: 100%|██████████| 50/50 [00:00<?, ?it/s]

Plotting labels to runs/detect/train16/labels.jpg... 


...


20 epochs completed in 0.177 hours.
Optimizer stripped from runs/detect/train16/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/train16/weights/best.pt, 6.2MB

Validating runs/detect/train16/weights/best.pt...
Ultralytics YOLOv8.1.43 🚀 Python-3.8.10 torch-2.0.1+cu117 CPU (11th Gen Intel Core(TM) i5-11320H 3.20GHz)
Model summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:05<00:00,  2.54s/it]


                   all         50         52      0.647      0.669      0.654      0.472
Speed: 1.1ms preprocess, 77.3ms inference, 0.0ms loss, 2.8ms postprocess per image
Results saved to runs/detect/train16


wandb: Waiting for W&B process to finish... (success).
wandb: 
wandb: Run history:
wandb:                  lr/pg0 ▁▂▄▅▆▆▇▇█████▇▆▅▄▃▂▁
wandb:                  lr/pg1 ▁▂▄▅▆▆▇▇█████▇▆▅▄▃▂▁
wandb:                  lr/pg2 ▁▂▄▅▆▆▇▇█████▇▆▅▄▃▂▁
wandb:        metrics/mAP50(B) ▆█▇▄▅▃▄▃▅▇▅▅▅▃▁▄▆▇▇█
wandb:     metrics/mAP50-95(B) ▇█▇▄▅▃▃▂▅▇▅▃▃▃▁▃▅▆▇█
wandb:    metrics/precision(B) ▁▇▁▁█▆▇▇▇▇▇▇▇▆▅▆▇▇█▇
wandb:       metrics/recall(B) █▅██▃▃▁▁▃▄▄▃▃▃▃▄▅▄▄▅
wandb:            model/GFLOPs ▁
wandb:        model/parameters ▁
wandb: model/speed_PyTorch(ms) ▁
wandb:          train/box_loss █▆▄▅▃▄▄▆▄▂▅▅▅▃▄▃▂▂▁▂
wandb:          train/cls_loss █▅▃▄▂▃▂▂▂▁▅▅▄▄▄▃▃▃▃▃
wandb:          train/dfl_loss █▃▃▂▂▂▃▃▃▁▅▄▆▃▄▅▁▁▂▄
wandb:            val/box_loss ▁▂▃▅▄▅▆▆▅▅▆█▇██▇▇▆▆▅
wandb:            val/cls_loss █▆▇█▆▆▇▇▅▃▅▇▆▆█▅▃▂▂▁
wandb:            val/dfl_loss ▁▂▃▄▄▄▅▅▅▅▆▇▆▇█▇▆▅▅▅
wandb: 
wandb: Run summary:
wandb:                  lr/pg0 0.00012
wandb:                  lr/pg1 0.00012
wandb:                  lr/pg2 0.000

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7fa46d34a310>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.048048, 

In [16]:
model_medium_100.train(data='YOLO_dataset1/config.yaml', epochs = 20)
#78.15%

New https://pypi.org/project/ultralytics/8.1.44 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.1.43 🚀 Python-3.8.10 torch-2.0.1+cu117 CPU (11th Gen Intel Core(TM) i5-11320H 3.20GHz)
engine/trainer: task=detect, mode=train, model=yolov8m.pt, data=YOLO_dataset1/config.yaml, epochs=20, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train17, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None,

wandb: wandb version 0.16.6 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
wandb: Tracking run with wandb version 0.15.12
wandb: Run data is saved locally in /home/vedansh/Desktop/Computer Vision/Assignment 4/wandb/run-20240407_235114-al4wypc4
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run train17
wandb: ⭐️ View project at https://wandb.ai/music-cognition/YOLOv8
wandb: 🚀 View run at https://wandb.ai/music-cognition/YOLOv8/runs/al4wypc4


Freezing layer 'model.22.dfl.conv.weight'


train: Scanning /home/vedansh/Desktop/Computer Vision/Assignment 4/YOLO_dataset1/labels/train.cache... 100 images, 0 backgrounds, 0 corrupt: 100%|██████████| 100/100 [00:00<?, ?it/s]
val: Scanning /home/vedansh/Desktop/Computer Vision/Assignment 4/YOLO_dataset1/labels/val.cache... 50 images, 0 backgrounds, 0 corrupt: 100%|██████████| 50/50 [00:00<?, ?it/s]

Plotting labels to runs/detect/train17/labels.jpg... 


...


20 epochs completed in 1.104 hours.
Optimizer stripped from runs/detect/train17/weights/last.pt, 52.0MB
Optimizer stripped from runs/detect/train17/weights/best.pt, 52.0MB

Validating runs/detect/train17/weights/best.pt...
Ultralytics YOLOv8.1.43 🚀 Python-3.8.10 torch-2.0.1+cu117 CPU (11th Gen Intel Core(TM) i5-11320H 3.20GHz)
Model summary (fused): 218 layers, 25840339 parameters, 0 gradients, 78.7 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:22<00:00, 11.42s/it]


                   all         50         52      0.765      0.712      0.782      0.685
Speed: 1.0ms preprocess, 431.1ms inference, 0.0ms loss, 2.7ms postprocess per image
Results saved to runs/detect/train17


wandb: Waiting for W&B process to finish... (success).
wandb: 
wandb: Run history:
wandb:                  lr/pg0 ▁▂▄▅▆▆▇▇█████▇▆▅▄▃▂▁
wandb:                  lr/pg1 ▁▂▄▅▆▆▇▇█████▇▆▅▄▃▂▁
wandb:                  lr/pg2 ▁▂▄▅▆▆▇▇█████▇▆▅▄▃▂▁
wandb:        metrics/mAP50(B) █▅▄▁▃▄▃▁▁▁▁▁▁▃▃▄▄▅▆█
wandb:     metrics/mAP50-95(B) █▄▃▁▂▃▂▁▁▁▁▁▁▂▂▃▃▃▄█
wandb:    metrics/precision(B) █▆▇▁▃▃▄▁▂▁▁▁▂▅▂▃▄▆▇█
wandb:       metrics/recall(B) █▇▄▆▅▆▃▃▁▂▃▆▇▃▇█▇▇▆█
wandb:            model/GFLOPs ▁
wandb:        model/parameters ▁
wandb: model/speed_PyTorch(ms) ▁
wandb:          train/box_loss ▃▃▁▂▁▇▆▇▇▄▇▇█▄▅▄▂▂▃▁
wandb:          train/cls_loss █▅▄▄▃▄▃▄▃▃▄▅▄▃▃▂▂▂▂▁
wandb:          train/dfl_loss ▃▁▁▁▂▄▃▅▄▃▅▆█▄▆▅▃▂▃▃
wandb:            val/box_loss ▁▂▄▃▄▄▅█▇█▇▆▅▄▄▄▄▄▃▃
wandb:            val/cls_loss ▁▁▁▂▁▁▂▇▃▇▇█▇▃▂▁▁▁▁▁
wandb:            val/dfl_loss ▁▁▁▂▂▂▃█▅█▆▆▄▂▂▂▂▂▁▁
wandb: 
wandb: Run summary:
wandb:                  lr/pg0 0.00012
wandb:                  lr/pg1 0.00012
wandb:                  lr/pg2 0.000

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7fa4506eba90>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.048048, 

Following are the mAP50 values that we obtain for all the above models that have been trained.

- Nano, Scratch, 400 samples: 24.63%
- Nano, Pretrained, 400 samples: 72.91%
- Medium, Pretrained, 400 samples: 70.77%
- Nano, Scratch, 100 samples: 2.1%
- Nano, Pretrained, 100 samples: 65.44%
- Medium, Pretrined, 100 samples: 78.15%

Increasing the size of the dataset increases accuracy only in the case where we train it from scratch whereas in the oth cases it doesnt matter as much since it is already pretrained. As for the size of the model, does not affect this much because of the basic nature of the task where we only need to detect one class of objects in this case the ducks. For more complicated tasks with more detections, we can predict that the medium model would do significantly better.

In [11]:
model_nano_pretrained_400 = YOLO('/home/vedansh/Desktop/Computer Vision/Assignment 4/runs/detect/train13/weights/best.torchscript')
model_nano_400 = YOLO('/home/vedansh/Desktop/Computer Vision/Assignment 4/runs/detect/train12/weights/best.pt')
model_medium_400 = YOLO('/home/vedansh/Desktop/Computer Vision/Assignment 4/runs/detect/train14/weights/best.pt')
model_nano_100 = YOLO('/home/vedansh/Desktop/Computer Vision/Assignment 4/runs/detect/train15/weights/best.pt')
model_nano_pretrained_100 = YOLO('/home/vedansh/Desktop/Computer Vision/Assignment 4/runs/detect/train16/weights/best.pt')
model_medium_100 = YOLO('/home/vedansh/Desktop/Computer Vision/Assignment 4/runs/detect/train17/weights/best.pt')

In [14]:
input = ['YOLO_dataset/images/val/0f5e9d02e8b110a5.png', 'YOLO_dataset/images/val/119d707a575b7721.png', 'YOLO_dataset/images/val/2d5dc410c4d5f31a.png', 'YOLO_dataset/images/val/327236c0ef04c7f0.png']
results = model_nano_pretrained_400(input)
for i, result in enumerate(results):
    result.show()


0: 640x640 1 duck, 74.3ms
1: 640x640 3 ducks, 74.3ms
2: 640x640 3 ducks, 74.3ms
3: 640x640 3 ducks, 74.3ms
Speed: 3.2ms preprocess, 74.3ms inference, 0.4ms postprocess per image at shape (1, 3, 640, 640)


### Part 5:

Here, when analysing the logoutputs during the training period, we notice there to be augmentations used by default. The augmentation used are:
- hsv_h: Adjusts the hue of the image by a fraction.
- hsv_s: Alters the saturation of the image by a fraction.
- hsv_v: Modifies the value (brightness) of the image by a fraction.
- translate: Translates the image horizontally and vertically by a fraction of the image size.
- scale: Scales the image by a gain factor.
- fliplr: Flips the image left to right with the specified probability.
- mosaic: Combines four training images into one.
- erasing: Randomly erases a portion of the image during classification training.
- crop_fraction: Crops the classification image to a fraction of its size.

In [16]:
model_nano_pretrained_100_noaugment = YOLO('yolov8n.pt')
model_nano_pretrained_100_noaugment.train(data='YOLO_dataset1/config.yaml', epochs=20, workers=5,
                                            mosaic=0.0, 
                                            mixup=0.0, 
                                            copy_paste=0.0,  
                                            auto_augment=None,  
                                            erasing=0.0, 
                                            hsv_h=0.0, hsv_s=0.0, hsv_v=0.0,  
                                            degrees=0.0, translate=0.0, scale=0.0, shear=0.0,  
                                            flipud=0.0, fliplr=0.0, 
                                            perspective=0.0,  
                                            bgr=0.0,  
                                            crop_fraction=0.0,
                                            )

Ultralytics YOLOv8.1.43 🚀 Python-3.8.10 torch-2.0.1+cu117 CPU (11th Gen Intel Core(TM) i5-11320H 3.20GHz)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=YOLO_dataset1/config.yaml, epochs=20, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=5, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, sho

wandb: Currently logged in as: vedu-agrawal (music-cognition). Use `wandb login --relogin` to force relogin
wandb: wandb version 0.16.6 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
wandb: Tracking run with wandb version 0.15.12
wandb: Run data is saved locally in /home/vedansh/Desktop/Computer Vision/Assignment 4/wandb/run-20240411_204414-xfx2m82c
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run train
wandb: ⭐️ View project at https://wandb.ai/music-cognition/YOLOv8
wandb: 🚀 View run at https://wandb.ai/music-cognition/YOLOv8/runs/xfx2m82c


Freezing layer 'model.22.dfl.conv.weight'


train: Scanning /home/vedansh/Desktop/Computer Vision/Assignment 4/YOLO_dataset1/labels/train.cache... 100 images, 0 backgrounds, 0 corrupt: 100%|██████████| 100/100 [00:00<?, ?it/s]
/home/vedansh/.local/lib/python3.8/site-packages/torch/cuda/__init__.py:546: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
val: Scanning /home/vedansh/Desktop/Computer Vision/Assignment 4/YOLO_dataset1/labels/val.cache... 50 images, 0 backgrounds, 0 corrupt: 100%|██████████| 50/50 [00:00<?, ?it/s]


Plotting labels to runs/detect/train/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.002, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 0 dataloader workers
Logging results to runs/detect/train
Starting training for 20 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...


20 epochs completed in 0.162 hours.
Optimizer stripped from runs/detect/train/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/train/weights/best.pt, 6.2MB

Validating runs/detect/train/weights/best.pt...
Ultralytics YOLOv8.1.43 🚀 Python-3.8.10 torch-2.0.1+cu117 CPU (11th Gen Intel Core(TM) i5-11320H 3.20GHz)
Model summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:04<00:00,  2.33s/it]


                   all         50         52      0.835      0.585      0.678       0.47
Speed: 1.0ms preprocess, 67.1ms inference, 0.0ms loss, 2.5ms postprocess per image
Results saved to runs/detect/train


wandb: Waiting for W&B process to finish... (success).
wandb: 
wandb: Run history:
wandb:                  lr/pg0 ▁▂▄▅▆▆▇▇█████▇▆▅▄▃▂▁
wandb:                  lr/pg1 ▁▂▄▅▆▆▇▇█████▇▆▅▄▃▂▁
wandb:                  lr/pg2 ▁▂▄▅▆▆▇▇█████▇▆▅▄▃▂▁
wandb:        metrics/mAP50(B) ▅▅▇▅▅▃▁▂▃▄▅▅▅▅▅▆▇█▇█
wandb:     metrics/mAP50-95(B) ▆▇█▆▅▄▁▃▃▄▅▆▆▆▆▇▇▇██
wandb:    metrics/precision(B) ▁▁▁▁▁▇▇█▆▆▆▆▇▆▆▆▆▇▇█
wandb:       metrics/recall(B) ████▇▁▂▂▄▄▄▄▄▅▅▅▅▅▅▅
wandb:            model/GFLOPs ▁
wandb:        model/parameters ▁
wandb: model/speed_PyTorch(ms) ▁
wandb:          train/box_loss █▆▅▄▃▃▃▃▃▂▂▂▂▂▂▂▁▁▁▁
wandb:          train/cls_loss █▆▅▄▄▃▃▃▃▂▂▂▂▂▁▁▁▁▁▁
wandb:          train/dfl_loss █▆▄▃▃▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁
wandb:            val/box_loss ▁▁▂▃▅▆█▇█▇▇▅▅▅▆▆▅▅▅▆
wandb:            val/cls_loss ▆▆▆▆▇▇█▇▆▅▄▄▃▂▂▂▂▂▁▁
wandb:            val/dfl_loss ▁▁▂▂▅▅▇▇██▆▅▅▅▅▅▅▅▅▅
wandb: 
wandb: Run summary:
wandb:                  lr/pg0 0.00012
wandb:                  lr/pg1 0.00012
wandb:                  lr/pg2 0.000

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7fc21f5d5220>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.048048, 

In [17]:
model_nano_pretrained_100_noaugment = YOLO('/home/vedansh/Desktop/Computer Vision/Assignment 4/runs/detect/train/weights/best.pt')

We can see that the mAP is 67.84 percent for the model without augmentations. This means that having augmentations gives better results. This is because with augmentations the model lears to detect ducks or objects with differing brightnesses and transalations etc. This way during inference it can detect objects with a different physical attribute from what was in the training data.

In order to find the augmentation which best affects the training, let us train the model by setting different parameters to 0 at a time and checking the mAP results for all of them.

In [18]:
model_nano_pretrained_100_noaugment = YOLO('yolov8n.pt')
model_nano_pretrained_100_noaugment.train(data='YOLO_dataset1/config.yaml', epochs=20, workers=5,
                                            mosaic=0.0
                                            )

Ultralytics YOLOv8.1.43 🚀 Python-3.8.10 torch-2.0.1+cu117 CPU (11th Gen Intel Core(TM) i5-11320H 3.20GHz)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=YOLO_dataset1/config.yaml, epochs=20, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=5, project=None, name=train2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, sh

wandb: wandb version 0.16.6 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
wandb: Tracking run with wandb version 0.15.12
wandb: Run data is saved locally in /home/vedansh/Desktop/Computer Vision/Assignment 4/wandb/run-20240411_210513-8s7x6kfd
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run train2
wandb: ⭐️ View project at https://wandb.ai/music-cognition/YOLOv8
wandb: 🚀 View run at https://wandb.ai/music-cognition/YOLOv8/runs/8s7x6kfd


Freezing layer 'model.22.dfl.conv.weight'


train: Scanning /home/vedansh/Desktop/Computer Vision/Assignment 4/YOLO_dataset1/labels/train.cache... 100 images, 0 backgrounds, 0 corrupt: 100%|██████████| 100/100 [00:00<?, ?it/s]
val: Scanning /home/vedansh/Desktop/Computer Vision/Assignment 4/YOLO_dataset1/labels/val.cache... 50 images, 0 backgrounds, 0 corrupt: 100%|██████████| 50/50 [00:00<?, ?it/s]

Plotting labels to runs/detect/train2/labels.jpg... 


...


20 epochs completed in 0.167 hours.
Optimizer stripped from runs/detect/train2/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/train2/weights/best.pt, 6.2MB

Validating runs/detect/train2/weights/best.pt...
Ultralytics YOLOv8.1.43 🚀 Python-3.8.10 torch-2.0.1+cu117 CPU (11th Gen Intel Core(TM) i5-11320H 3.20GHz)
Model summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:04<00:00,  2.22s/it]


                   all         50         52      0.799      0.635      0.734      0.513
Speed: 1.0ms preprocess, 62.9ms inference, 0.0ms loss, 3.7ms postprocess per image
Results saved to runs/detect/train2


wandb: Waiting for W&B process to finish... (success).
wandb: 
wandb: Run history:
wandb:                  lr/pg0 ▁▂▄▅▆▆▇▇█████▇▆▅▄▃▂▁
wandb:                  lr/pg1 ▁▂▄▅▆▆▇▇█████▇▆▅▄▃▂▁
wandb:                  lr/pg2 ▁▂▄▅▆▆▇▇█████▇▆▅▄▃▂▁
wandb:        metrics/mAP50(B) ▅▆▅▄▄▂▄▄▄▁▃▃▄▄▄▆▇███
wandb:     metrics/mAP50-95(B) ▆▇▅▅▅▃▃▃▄▁▂▄▄▄▄▅▇▇▇█
wandb:    metrics/precision(B) ▁▁▁▇█▆█▇▆▄▅▆▆▆▆▆▇▇▇▇
wandb:       metrics/recall(B) ███▁▂▂▂▃▄▃▃▄▄▄▄▅▅▅▅▅
wandb:            model/GFLOPs ▁
wandb:        model/parameters ▁
wandb: model/speed_PyTorch(ms) ▁
wandb:          train/box_loss █▄▄▄▃▆▄▃▅▄▄▄▄▅▆▃▂▂▂▁
wandb:          train/cls_loss █▅▅▄▃▃▄▃▄▃▃▂▂▃▃▂▁▁▁▁
wandb:          train/dfl_loss █▆▅▆▅▆▅▃▆▆▆▄▅▆▅▄▃▂▃▁
wandb:            val/box_loss ▁▂▃▃▃▄▅▅▆█▇▆▆▆▆▅▅▅▅▅
wandb:            val/cls_loss ████▇██▇▆▇▆▅▄▄▄▃▂▁▁▁
wandb:            val/dfl_loss ▁▂▃▃▃▅▅▆▇█▆▇██▇▆▅▅▅▅
wandb: 
wandb: Run summary:
wandb:                  lr/pg0 0.00012
wandb:                  lr/pg1 0.00012
wandb:                  lr/pg2 0.000

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7fc218183a00>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.048048, 

In [19]:
model_nano_pretrained_100_noaugment = YOLO('yolov8n.pt')
model_nano_pretrained_100_noaugment.train(data='YOLO_dataset1/config.yaml', epochs=20, workers=5, 
                                            erasing=0.0
                                            )

Ultralytics YOLOv8.1.43 🚀 Python-3.8.10 torch-2.0.1+cu117 CPU (11th Gen Intel Core(TM) i5-11320H 3.20GHz)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=YOLO_dataset1/config.yaml, epochs=20, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=5, project=None, name=train3, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, sh

wandb: wandb version 0.16.6 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
wandb: Tracking run with wandb version 0.15.12
wandb: Run data is saved locally in /home/vedansh/Desktop/Computer Vision/Assignment 4/wandb/run-20240411_211609-euov1qbu
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run train3
wandb: ⭐️ View project at https://wandb.ai/music-cognition/YOLOv8
wandb: 🚀 View run at https://wandb.ai/music-cognition/YOLOv8/runs/euov1qbu


Freezing layer 'model.22.dfl.conv.weight'


train: Scanning /home/vedansh/Desktop/Computer Vision/Assignment 4/YOLO_dataset1/labels/train.cache... 100 images, 0 backgrounds, 0 corrupt: 100%|██████████| 100/100 [00:00<?, ?it/s]
val: Scanning /home/vedansh/Desktop/Computer Vision/Assignment 4/YOLO_dataset1/labels/val.cache... 50 images, 0 backgrounds, 0 corrupt: 100%|██████████| 50/50 [00:00<?, ?it/s]

Plotting labels to runs/detect/train3/labels.jpg... 


...


20 epochs completed in 0.185 hours.
Optimizer stripped from runs/detect/train3/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/train3/weights/best.pt, 6.2MB

Validating runs/detect/train3/weights/best.pt...
Ultralytics YOLOv8.1.43 🚀 Python-3.8.10 torch-2.0.1+cu117 CPU (11th Gen Intel Core(TM) i5-11320H 3.20GHz)
Model summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:05<00:00,  2.58s/it]


                   all         50         52      0.647      0.669      0.654      0.472
Speed: 1.0ms preprocess, 79.2ms inference, 0.0ms loss, 2.4ms postprocess per image
Results saved to runs/detect/train3


wandb: Waiting for W&B process to finish... (success).
wandb: 
wandb: Run history:
wandb:                  lr/pg0 ▁▂▄▅▆▆▇▇█████▇▆▅▄▃▂▁
wandb:                  lr/pg1 ▁▂▄▅▆▆▇▇█████▇▆▅▄▃▂▁
wandb:                  lr/pg2 ▁▂▄▅▆▆▇▇█████▇▆▅▄▃▂▁
wandb:        metrics/mAP50(B) ▆█▇▄▅▃▄▃▅▇▅▅▅▃▁▄▆▇▇█
wandb:     metrics/mAP50-95(B) ▇█▇▄▅▃▃▂▅▇▅▃▃▃▁▃▅▆▇█
wandb:    metrics/precision(B) ▁▇▁▁█▆▇▇▇▇▇▇▇▆▅▆▇▇█▇
wandb:       metrics/recall(B) █▅██▃▃▁▁▃▄▄▃▃▃▃▄▅▄▄▅
wandb:            model/GFLOPs ▁
wandb:        model/parameters ▁
wandb: model/speed_PyTorch(ms) ▁
wandb:          train/box_loss █▆▄▅▃▄▄▆▄▂▅▅▅▃▄▃▂▂▁▂
wandb:          train/cls_loss █▅▃▄▂▃▂▂▂▁▅▅▄▄▄▃▃▃▃▃
wandb:          train/dfl_loss █▃▃▂▂▂▃▃▃▁▅▄▆▃▄▅▁▁▂▄
wandb:            val/box_loss ▁▂▃▅▄▅▆▆▅▅▆█▇██▇▇▆▆▅
wandb:            val/cls_loss █▆▇█▆▆▇▇▅▃▅▇▆▆█▅▃▂▂▁
wandb:            val/dfl_loss ▁▂▃▄▄▄▅▅▅▅▆▇▆▇█▇▆▅▅▅
wandb: 
wandb: Run summary:
wandb:                  lr/pg0 0.00012
wandb:                  lr/pg1 0.00012
wandb:                  lr/pg2 0.000

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7fc3550bd1c0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.048048, 

In [20]:
model_nano_pretrained_100_noaugment = YOLO('yolov8n.pt')
model_nano_pretrained_100_noaugment.train(data='YOLO_dataset1/config.yaml', epochs=20, workers=5, 
                                            hsv_h=0.0, hsv_s=0.0, hsv_v=0.0
                                            )

Ultralytics YOLOv8.1.43 🚀 Python-3.8.10 torch-2.0.1+cu117 CPU (11th Gen Intel Core(TM) i5-11320H 3.20GHz)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=YOLO_dataset1/config.yaml, epochs=20, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=5, project=None, name=train4, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, sh

wandb: wandb version 0.16.6 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
wandb: Tracking run with wandb version 0.15.12
wandb: Run data is saved locally in /home/vedansh/Desktop/Computer Vision/Assignment 4/wandb/run-20240411_212802-3vw5b8ft
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run train4
wandb: ⭐️ View project at https://wandb.ai/music-cognition/YOLOv8
wandb: 🚀 View run at https://wandb.ai/music-cognition/YOLOv8/runs/3vw5b8ft


Freezing layer 'model.22.dfl.conv.weight'


train: Scanning /home/vedansh/Desktop/Computer Vision/Assignment 4/YOLO_dataset1/labels/train.cache... 100 images, 0 backgrounds, 0 corrupt: 100%|██████████| 100/100 [00:00<?, ?it/s]
val: Scanning /home/vedansh/Desktop/Computer Vision/Assignment 4/YOLO_dataset1/labels/val.cache... 50 images, 0 backgrounds, 0 corrupt: 100%|██████████| 50/50 [00:00<?, ?it/s]

Plotting labels to runs/detect/train4/labels.jpg... 


...


20 epochs completed in 0.180 hours.
Optimizer stripped from runs/detect/train4/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/train4/weights/best.pt, 6.2MB

Validating runs/detect/train4/weights/best.pt...
Ultralytics YOLOv8.1.43 🚀 Python-3.8.10 torch-2.0.1+cu117 CPU (11th Gen Intel Core(TM) i5-11320H 3.20GHz)
Model summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:05<00:00,  2.90s/it]


                   all         50         52    0.00333      0.962      0.746      0.541
Speed: 1.0ms preprocess, 86.1ms inference, 0.0ms loss, 3.0ms postprocess per image
Results saved to runs/detect/train4


wandb: Waiting for W&B process to finish... (success).
wandb: 
wandb: Run history:
wandb:                  lr/pg0 ▁▂▄▅▆▆▇▇█████▇▆▅▄▃▂▁
wandb:                  lr/pg1 ▁▂▄▅▆▆▇▇█████▇▆▅▄▃▂▁
wandb:                  lr/pg2 ▁▂▄▅▆▆▇▇█████▇▆▅▄▃▂▁
wandb:        metrics/mAP50(B) ▆▇█▆▅▄▅▄▂▂▁▁▂▃▃▃▄▆▆█
wandb:     metrics/mAP50-95(B) ▆▆█▆▅▄▄▃▁▂▁▁▂▂▂▃▄▅▆█
wandb:    metrics/precision(B) ▁▇▁▁▆▅█▆▅▃▃▃▃▄▄▅▅▅▆▁
wandb:       metrics/recall(B) █▂██▂▃▁▁▃▂▂▂▂▃▃▄▄▅▄█
wandb:            model/GFLOPs ▁
wandb:        model/parameters ▁
wandb: model/speed_PyTorch(ms) ▁
wandb:          train/box_loss █▆▄▄▃▃▅▇▃▃▃▄▄▄▃▄▂▁▂▁
wandb:          train/cls_loss █▅▃▃▂▃▂▂▁▁▅▅▄▄▄▃▃▃▃▃
wandb:          train/dfl_loss █▃▃▁▁▂▄▄▃▂▄▄▅▄▃▅▂▁▅▂
wandb:            val/box_loss ▁▁▂▂▃▄▅▅█▆▇▇▇▆▇▇▅▅▄▄
wandb:            val/cls_loss ▅▄▅▅▅▅▆▆▇▆▇█▇▆▅▄▃▂▁▁
wandb:            val/dfl_loss ▁▁▁▂▂▃▃▃█▆▇▆▅▅▆▅▄▄▃▃
wandb: 
wandb: Run summary:
wandb:                  lr/pg0 0.00012
wandb:                  lr/pg1 0.00012
wandb:                  lr/pg2 0.000

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7fc294ad43d0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.048048, 

In [21]:
model_nano_pretrained_100_noaugment = YOLO('yolov8n.pt')
model_nano_pretrained_100_noaugment.train(data='YOLO_dataset1/config.yaml', epochs=20, workers=5,  
                                            degrees=0.0, translate=0.0, scale=0.0, shear=0.0
                                            )

Ultralytics YOLOv8.1.43 🚀 Python-3.8.10 torch-2.0.1+cu117 CPU (11th Gen Intel Core(TM) i5-11320H 3.20GHz)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=YOLO_dataset1/config.yaml, epochs=20, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=5, project=None, name=train5, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, sh

wandb: wandb version 0.16.6 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
wandb: Tracking run with wandb version 0.15.12
wandb: Run data is saved locally in /home/vedansh/Desktop/Computer Vision/Assignment 4/wandb/run-20240411_213941-atjd6yjm
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run train5
wandb: ⭐️ View project at https://wandb.ai/music-cognition/YOLOv8
wandb: 🚀 View run at https://wandb.ai/music-cognition/YOLOv8/runs/atjd6yjm


Freezing layer 'model.22.dfl.conv.weight'


train: Scanning /home/vedansh/Desktop/Computer Vision/Assignment 4/YOLO_dataset1/labels/train.cache... 100 images, 0 backgrounds, 0 corrupt: 100%|██████████| 100/100 [00:00<?, ?it/s]
val: Scanning /home/vedansh/Desktop/Computer Vision/Assignment 4/YOLO_dataset1/labels/val.cache... 50 images, 0 backgrounds, 0 corrupt: 100%|██████████| 50/50 [00:00<?, ?it/s]

Plotting labels to runs/detect/train5/labels.jpg... 


...


20 epochs completed in 0.175 hours.
Optimizer stripped from runs/detect/train5/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/train5/weights/best.pt, 6.2MB

Validating runs/detect/train5/weights/best.pt...
Ultralytics YOLOv8.1.43 🚀 Python-3.8.10 torch-2.0.1+cu117 CPU (11th Gen Intel Core(TM) i5-11320H 3.20GHz)
Model summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:05<00:00,  2.52s/it]


                   all         50         52     0.0034      0.981      0.675      0.534
Speed: 1.0ms preprocess, 76.7ms inference, 0.0ms loss, 2.7ms postprocess per image
Results saved to runs/detect/train5


wandb: Waiting for W&B process to finish... (success).
wandb: 
wandb: Run history:
wandb:                  lr/pg0 ▁▂▄▅▆▆▇▇█████▇▆▅▄▃▂▁
wandb:                  lr/pg1 ▁▂▄▅▆▆▇▇█████▇▆▅▄▃▂▁
wandb:                  lr/pg2 ▁▂▄▅▆▆▇▇█████▇▆▅▄▃▂▁
wandb:        metrics/mAP50(B) ▇█▇▂▃▄▄▂▅▄▄▁▂▃▃▃▅▅▇█
wandb:     metrics/mAP50-95(B) ▆█▇▃▄▃▄▃▅▃▃▁▂▃▃▃▃▅▆█
wandb:    metrics/precision(B) ▁▁▁▁▄▄█▅█▆▇▇▆▆▅▆▇▇▇▁
wandb:       metrics/recall(B) ████▄▅▁▁▃▃▂▂▂▃▃▃▃▄▄█
wandb:            model/GFLOPs ▁
wandb:        model/parameters ▁
wandb: model/speed_PyTorch(ms) ▁
wandb:          train/box_loss █▇▇▆▆▆▆▆▅▅▅▄▄▃▃▂▂▂▁▁
wandb:          train/cls_loss █▆▄▅▃▄▄▄▃▃▅▅▄▃▃▂▂▂▁▁
wandb:          train/dfl_loss █▆▇▅▅▅▅▅▅▅▆▄▃▂▂▂▁▁▁▁
wandb:            val/box_loss ▁▁▂▄▃▅▄▄▅▅▆██▇▆▅▅▅▄▄
wandb:            val/cls_loss ▆▅▆▇▆▇▆▅▅▆▇██▆▅▅▄▃▂▁
wandb:            val/dfl_loss ▁▁▂▃▃▄▃▄▆▅▆██▆▅▅▅▄▃▃
wandb: 
wandb: Run summary:
wandb:                  lr/pg0 0.00012
wandb:                  lr/pg1 0.00012
wandb:                  lr/pg2 0.000

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7fc3551219d0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.048048, 

In [22]:
model_nano_pretrained_100_noaugment = YOLO('yolov8n.pt')
model_nano_pretrained_100_noaugment.train(data='YOLO_dataset1/config.yaml', epochs=20, workers=5,  
                                            flipud=0.0, fliplr=0.0
                                            )

Ultralytics YOLOv8.1.43 🚀 Python-3.8.10 torch-2.0.1+cu117 CPU (11th Gen Intel Core(TM) i5-11320H 3.20GHz)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=YOLO_dataset1/config.yaml, epochs=20, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=5, project=None, name=train6, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, sh

wandb: wandb version 0.16.6 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
wandb: Tracking run with wandb version 0.15.12
wandb: Run data is saved locally in /home/vedansh/Desktop/Computer Vision/Assignment 4/wandb/run-20240411_215053-9sgedc0z
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run train6
wandb: ⭐️ View project at https://wandb.ai/music-cognition/YOLOv8
wandb: 🚀 View run at https://wandb.ai/music-cognition/YOLOv8/runs/9sgedc0z


Freezing layer 'model.22.dfl.conv.weight'


train: Scanning /home/vedansh/Desktop/Computer Vision/Assignment 4/YOLO_dataset1/labels/train.cache... 100 images, 0 backgrounds, 0 corrupt: 100%|██████████| 100/100 [00:00<?, ?it/s]
val: Scanning /home/vedansh/Desktop/Computer Vision/Assignment 4/YOLO_dataset1/labels/val.cache... 50 images, 0 backgrounds, 0 corrupt: 100%|██████████| 50/50 [00:00<?, ?it/s]

Plotting labels to runs/detect/train6/labels.jpg... 


...


20 epochs completed in 0.177 hours.
Optimizer stripped from runs/detect/train6/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/train6/weights/best.pt, 6.2MB

Validating runs/detect/train6/weights/best.pt...
Ultralytics YOLOv8.1.43 🚀 Python-3.8.10 torch-2.0.1+cu117 CPU (11th Gen Intel Core(TM) i5-11320H 3.20GHz)
Model summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:04<00:00,  2.46s/it]


                   all         50         52      0.848      0.429      0.653      0.485
Speed: 1.0ms preprocess, 73.9ms inference, 0.0ms loss, 2.5ms postprocess per image
Results saved to runs/detect/train6


wandb: Waiting for W&B process to finish... (success).
wandb: 
wandb: Run history:
wandb:                  lr/pg0 ▁▂▄▅▆▆▇▇█████▇▆▅▄▃▂▁
wandb:                  lr/pg1 ▁▂▄▅▆▆▇▇█████▇▆▅▄▃▂▁
wandb:                  lr/pg2 ▁▂▄▅▆▆▇▇█████▇▆▅▄▃▂▁
wandb:        metrics/mAP50(B) ▇█▇▆▂▇▅▆▆▄▁▅▆▅▆▇▇▇██
wandb:     metrics/mAP50-95(B) ▇█▇▅▃▅▅▅▆▄▁▄▅▄▅▆▆▇▇█
wandb:    metrics/precision(B) ▁█▁▁▄▆▆▆▆▅▄▆▇▆▇▆▆▇▇█
wandb:       metrics/recall(B) █▄██▁▄▃▄▄▄▃▃▄▅▄▄▅▄▅▄
wandb:            model/GFLOPs ▁
wandb:        model/parameters ▁
wandb: model/speed_PyTorch(ms) ▁
wandb:          train/box_loss █▇▅▅▄▅▅▅▄▃▄▄▃▂▃▃▂▂▁▁
wandb:          train/cls_loss █▅▃▄▃▃▂▂▂▁▄▄▄▃▃▂▃▂▂▂
wandb:          train/dfl_loss █▅▄▃▃▃▄▃▃▃▂▄▃▂▄▃▁▁▂▂
wandb:            val/box_loss ▁▂▂▃▅▄▄▃▄▅█▆▅▅▅▄▄▄▃▃
wandb:            val/cls_loss ▆▅▅▆▆▄▅▄▄▅█▆▅▄▃▃▃▂▁▁
wandb:            val/dfl_loss ▁▂▂▃▄▄▃▃▄▅█▅▅▅▄▄▄▃▃▃
wandb: 
wandb: Run summary:
wandb:                  lr/pg0 0.00012
wandb:                  lr/pg1 0.00012
wandb:                  lr/pg2 0.000

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7fc2196e9c70>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.048048, 

In [23]:
model_nano_pretrained_100_noaugment = YOLO('yolov8n.pt')
model_nano_pretrained_100_noaugment.train(data='YOLO_dataset1/config.yaml', epochs=20, workers=5, 
                                            perspective=0.0
                                            )

Ultralytics YOLOv8.1.43 🚀 Python-3.8.10 torch-2.0.1+cu117 CPU (11th Gen Intel Core(TM) i5-11320H 3.20GHz)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=YOLO_dataset1/config.yaml, epochs=20, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=5, project=None, name=train7, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, sh

wandb: wandb version 0.16.6 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
wandb: Tracking run with wandb version 0.15.12
wandb: Run data is saved locally in /home/vedansh/Desktop/Computer Vision/Assignment 4/wandb/run-20240411_220219-2ujkjvec
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run train7
wandb: ⭐️ View project at https://wandb.ai/music-cognition/YOLOv8
wandb: 🚀 View run at https://wandb.ai/music-cognition/YOLOv8/runs/2ujkjvec


Freezing layer 'model.22.dfl.conv.weight'


train: Scanning /home/vedansh/Desktop/Computer Vision/Assignment 4/YOLO_dataset1/labels/train.cache... 100 images, 0 backgrounds, 0 corrupt: 100%|██████████| 100/100 [00:00<?, ?it/s]
val: Scanning /home/vedansh/Desktop/Computer Vision/Assignment 4/YOLO_dataset1/labels/val.cache... 50 images, 0 backgrounds, 0 corrupt: 100%|██████████| 50/50 [00:00<?, ?it/s]

Plotting labels to runs/detect/train7/labels.jpg... 


...


20 epochs completed in 0.179 hours.
Optimizer stripped from runs/detect/train7/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/train7/weights/best.pt, 6.2MB

Validating runs/detect/train7/weights/best.pt...
Ultralytics YOLOv8.1.43 🚀 Python-3.8.10 torch-2.0.1+cu117 CPU (11th Gen Intel Core(TM) i5-11320H 3.20GHz)
Model summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:05<00:00,  2.51s/it]


                   all         50         52      0.647      0.669      0.654      0.472
Speed: 1.0ms preprocess, 76.2ms inference, 0.0ms loss, 2.8ms postprocess per image
Results saved to runs/detect/train7


wandb: Waiting for W&B process to finish... (success).
wandb: 
wandb: Run history:
wandb:                  lr/pg0 ▁▂▄▅▆▆▇▇█████▇▆▅▄▃▂▁
wandb:                  lr/pg1 ▁▂▄▅▆▆▇▇█████▇▆▅▄▃▂▁
wandb:                  lr/pg2 ▁▂▄▅▆▆▇▇█████▇▆▅▄▃▂▁
wandb:        metrics/mAP50(B) ▆█▇▄▅▃▄▃▅▇▅▅▅▃▁▄▆▇▇█
wandb:     metrics/mAP50-95(B) ▇█▇▄▅▃▃▂▅▇▅▃▃▃▁▃▅▆▇█
wandb:    metrics/precision(B) ▁▇▁▁█▆▇▇▇▇▇▇▇▆▅▆▇▇█▇
wandb:       metrics/recall(B) █▅██▃▃▁▁▃▄▄▃▃▃▃▄▅▄▄▅
wandb:            model/GFLOPs ▁
wandb:        model/parameters ▁
wandb: model/speed_PyTorch(ms) ▁
wandb:          train/box_loss █▆▄▅▃▄▄▆▄▂▅▅▅▃▄▃▂▂▁▂
wandb:          train/cls_loss █▅▃▄▂▃▂▂▂▁▅▅▄▄▄▃▃▃▃▃
wandb:          train/dfl_loss █▃▃▂▂▂▃▃▃▁▅▄▆▃▄▅▁▁▂▄
wandb:            val/box_loss ▁▂▃▅▄▅▆▆▅▅▆█▇██▇▇▆▆▅
wandb:            val/cls_loss █▆▇█▆▆▇▇▅▃▅▇▆▆█▅▃▂▂▁
wandb:            val/dfl_loss ▁▂▃▄▄▄▅▅▅▅▆▇▆▇█▇▆▅▅▅
wandb: 
wandb: Run summary:
wandb:                  lr/pg0 0.00012
wandb:                  lr/pg1 0.00012
wandb:                  lr/pg2 0.000

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7fc20d613d60>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.048048, 

In [24]:
model_nano_pretrained_100_noaugment = YOLO('yolov8n.pt')
model_nano_pretrained_100_noaugment.train(data='YOLO_dataset1/config.yaml', epochs=20, workers=5,  
                                            bgr=0.0
                                            )

Ultralytics YOLOv8.1.43 🚀 Python-3.8.10 torch-2.0.1+cu117 CPU (11th Gen Intel Core(TM) i5-11320H 3.20GHz)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=YOLO_dataset1/config.yaml, epochs=20, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=5, project=None, name=train8, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, sh

wandb: wandb version 0.16.6 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
wandb: Tracking run with wandb version 0.15.12
wandb: Run data is saved locally in /home/vedansh/Desktop/Computer Vision/Assignment 4/wandb/run-20240411_221355-9tjbhih8
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run train8
wandb: ⭐️ View project at https://wandb.ai/music-cognition/YOLOv8
wandb: 🚀 View run at https://wandb.ai/music-cognition/YOLOv8/runs/9tjbhih8


Freezing layer 'model.22.dfl.conv.weight'


train: Scanning /home/vedansh/Desktop/Computer Vision/Assignment 4/YOLO_dataset1/labels/train.cache... 100 images, 0 backgrounds, 0 corrupt: 100%|██████████| 100/100 [00:00<?, ?it/s]
val: Scanning /home/vedansh/Desktop/Computer Vision/Assignment 4/YOLO_dataset1/labels/val.cache... 50 images, 0 backgrounds, 0 corrupt: 100%|██████████| 50/50 [00:00<?, ?it/s]

Plotting labels to runs/detect/train8/labels.jpg... 


...


20 epochs completed in 0.178 hours.
Optimizer stripped from runs/detect/train8/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/train8/weights/best.pt, 6.2MB

Validating runs/detect/train8/weights/best.pt...
Ultralytics YOLOv8.1.43 🚀 Python-3.8.10 torch-2.0.1+cu117 CPU (11th Gen Intel Core(TM) i5-11320H 3.20GHz)
Model summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:04<00:00,  2.26s/it]


                   all         50         52      0.647      0.669      0.654      0.472
Speed: 1.0ms preprocess, 65.6ms inference, 0.0ms loss, 2.2ms postprocess per image
Results saved to runs/detect/train8


wandb: Waiting for W&B process to finish... (success).
wandb: 
wandb: Run history:
wandb:                  lr/pg0 ▁▂▄▅▆▆▇▇█████▇▆▅▄▃▂▁
wandb:                  lr/pg1 ▁▂▄▅▆▆▇▇█████▇▆▅▄▃▂▁
wandb:                  lr/pg2 ▁▂▄▅▆▆▇▇█████▇▆▅▄▃▂▁
wandb:        metrics/mAP50(B) ▆█▇▄▅▃▄▃▅▇▅▅▅▃▁▄▆▇▇█
wandb:     metrics/mAP50-95(B) ▇█▇▄▅▃▃▂▅▇▅▃▃▃▁▃▅▆▇█
wandb:    metrics/precision(B) ▁▇▁▁█▆▇▇▇▇▇▇▇▆▅▆▇▇█▇
wandb:       metrics/recall(B) █▅██▃▃▁▁▃▄▄▃▃▃▃▄▅▄▄▅
wandb:            model/GFLOPs ▁
wandb:        model/parameters ▁
wandb: model/speed_PyTorch(ms) ▁
wandb:          train/box_loss █▆▄▅▃▄▄▆▄▂▅▅▅▃▄▃▂▂▁▂
wandb:          train/cls_loss █▅▃▄▂▃▂▂▂▁▅▅▄▄▄▃▃▃▃▃
wandb:          train/dfl_loss █▃▃▂▂▂▃▃▃▁▅▄▆▃▄▅▁▁▂▄
wandb:            val/box_loss ▁▂▃▅▄▅▆▆▅▅▆█▇██▇▇▆▆▅
wandb:            val/cls_loss █▆▇█▆▆▇▇▅▃▅▇▆▆█▅▃▂▂▁
wandb:            val/dfl_loss ▁▂▃▄▄▄▅▅▅▅▆▇▆▇█▇▆▅▅▅
wandb: 
wandb: Run summary:
wandb:                  lr/pg0 0.00012
wandb:                  lr/pg1 0.00012
wandb:                  lr/pg2 0.000

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7fc21f5f77c0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.048048, 

In [25]:
model_nano_pretrained_100_noaugment = YOLO('yolov8n.pt')
model_nano_pretrained_100_noaugment.train(data='YOLO_dataset1/config.yaml', epochs=20, workers=5, 
                                            crop_fraction=0.0
                                            )

Ultralytics YOLOv8.1.43 🚀 Python-3.8.10 torch-2.0.1+cu117 CPU (11th Gen Intel Core(TM) i5-11320H 3.20GHz)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=YOLO_dataset1/config.yaml, epochs=20, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=5, project=None, name=train9, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, sh

wandb: wandb version 0.16.6 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
wandb: Tracking run with wandb version 0.15.12
wandb: Run data is saved locally in /home/vedansh/Desktop/Computer Vision/Assignment 4/wandb/run-20240411_222524-8h416x8e
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run train9
wandb: ⭐️ View project at https://wandb.ai/music-cognition/YOLOv8
wandb: 🚀 View run at https://wandb.ai/music-cognition/YOLOv8/runs/8h416x8e


Freezing layer 'model.22.dfl.conv.weight'


train: Scanning /home/vedansh/Desktop/Computer Vision/Assignment 4/YOLO_dataset1/labels/train.cache... 100 images, 0 backgrounds, 0 corrupt: 100%|██████████| 100/100 [00:00<?, ?it/s]
val: Scanning /home/vedansh/Desktop/Computer Vision/Assignment 4/YOLO_dataset1/labels/val.cache... 50 images, 0 backgrounds, 0 corrupt: 100%|██████████| 50/50 [00:00<?, ?it/s]

Plotting labels to runs/detect/train9/labels.jpg... 


...


20 epochs completed in 0.187 hours.
Optimizer stripped from runs/detect/train9/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/train9/weights/best.pt, 6.2MB

Validating runs/detect/train9/weights/best.pt...
Ultralytics YOLOv8.1.43 🚀 Python-3.8.10 torch-2.0.1+cu117 CPU (11th Gen Intel Core(TM) i5-11320H 3.20GHz)
Model summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:05<00:00,  2.56s/it]


                   all         50         52      0.647      0.669      0.654      0.472
Speed: 1.0ms preprocess, 78.2ms inference, 0.0ms loss, 2.6ms postprocess per image
Results saved to runs/detect/train9


wandb: Waiting for W&B process to finish... (success).
wandb: 
wandb: Run history:
wandb:                  lr/pg0 ▁▂▄▅▆▆▇▇█████▇▆▅▄▃▂▁
wandb:                  lr/pg1 ▁▂▄▅▆▆▇▇█████▇▆▅▄▃▂▁
wandb:                  lr/pg2 ▁▂▄▅▆▆▇▇█████▇▆▅▄▃▂▁
wandb:        metrics/mAP50(B) ▆█▇▄▅▃▄▃▅▇▅▅▅▃▁▄▆▇▇█
wandb:     metrics/mAP50-95(B) ▇█▇▄▅▃▃▂▅▇▅▃▃▃▁▃▅▆▇█
wandb:    metrics/precision(B) ▁▇▁▁█▆▇▇▇▇▇▇▇▆▅▆▇▇█▇
wandb:       metrics/recall(B) █▅██▃▃▁▁▃▄▄▃▃▃▃▄▅▄▄▅
wandb:            model/GFLOPs ▁
wandb:        model/parameters ▁
wandb: model/speed_PyTorch(ms) ▁
wandb:          train/box_loss █▆▄▅▃▄▄▆▄▂▅▅▅▃▄▃▂▂▁▂
wandb:          train/cls_loss █▅▃▄▂▃▂▂▂▁▅▅▄▄▄▃▃▃▃▃
wandb:          train/dfl_loss █▃▃▂▂▂▃▃▃▁▅▄▆▃▄▅▁▁▂▄
wandb:            val/box_loss ▁▂▃▅▄▅▆▆▅▅▆█▇██▇▇▆▆▅
wandb:            val/cls_loss █▆▇█▆▆▇▇▅▃▅▇▆▆█▅▃▂▂▁
wandb:            val/dfl_loss ▁▂▃▄▄▄▅▅▅▅▆▇▆▇█▇▆▅▅▅
wandb: 
wandb: Run summary:
wandb:                  lr/pg0 0.00012
wandb:                  lr/pg1 0.00012
wandb:                  lr/pg2 0.000

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7fc20d7b5f40>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.048048, 

From the above, we can tell that Mosaicing would be a very important augmentation since it allows the model to learn to detect even partial duck images which makes it more better at the detection task than any other augmentation would.